In [40]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [41]:
train = pd.read_csv("DS_Emp.csv")

In [42]:
train.shape

(19158, 14)

In [43]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
enrollee_id,19158.0,16875.358179,9616.292592,1.000,8554.25,16982.500,25169.75,33380.000
city_development_index,19158.0,0.828848,0.123362,0.448,0.74,0.903,0.92,0.949
training_hours,19158.0,65.366896,60.058462,1.000,23.00,47.000,88.00,336.000
target,19158.0,0.249348,0.432647,0.000,0.00,0.000,0.00,1.000


In [44]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

In [45]:
train.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0


In [46]:
train[['relevent_experience','experience','last_new_job']].sample(15)

,relevent_experience,experience,last_new_job
18226,Has relevent experience,>20,3
2937,Has relevent experience,15,>4
3179,Has relevent experience,19,1
5215,Has relevent experience,10,4
1588,No relevent experience,1,1
6671,Has relevent experience,4,never
17181,Has relevent experience,1,1
9515,Has relevent experience,19,1
7567,Has relevent experience,2,1
3181,Has relevent experience,3,1


In [9]:
##preprocessing

In [47]:
from sklearn.preprocessing import LabelEncoder

In [48]:
train['enrolled_university'].value_counts()

enrolled_university
no_enrollment       13817
Full time course     3757
Part time course     1198
Name: count, dtype: int64

In [49]:
# Forward-fill safely (no chained assignment)
train.loc[:, 'gender'] = train['gender'].ffill()
# If the first rows are still NaN after ffill, also back-fill as a fallback:
train.loc[:, 'gender'] = train['gender'].ffill().bfill()



In [50]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

cat_cols = ['enrolled_university',  # add your other categorical cols here
            # ...
           ]
num_cols = [ # your numeric cols
           ]

categorical_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

numeric_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

preprocessor = ColumnTransformer([
    ("cat", categorical_tf, cat_cols),
    ("num", numeric_tf, num_cols)
])

clf = Pipeline([
    ("prep", preprocessor),
    ("rf", RandomForestClassifier(class_weight="balanced", n_estimators=300, random_state=42))
])

clf.fit(train.drop(columns=['target']), train['target'])
pred = clf.predict(test)  # works even if test has unseen categories


In [51]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

cat_cols = ['education_level',  # plus your other categoricals
            # ...
           ]
num_cols = [ # your numeric cols
           ]

categorical_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

numeric_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

preprocessor = ColumnTransformer([
    ("cat", categorical_tf, cat_cols),
    ("num", numeric_tf, num_cols)
])

model = Pipeline([
    ("prep", preprocessor),
    ("rf", RandomForestClassifier(class_weight="balanced", random_state=42))
])


In [52]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

cat_cols = ['major_discipline',  # add other categoricals here
            # ...
           ]
num_cols = [ # your numeric columns
           ]

categorical_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

numeric_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

preprocessor = ColumnTransformer([
    ("cat", categorical_tf, cat_cols),
    ("num", numeric_tf, num_cols)
])

model = Pipeline([
    ("prep", preprocessor),
    ("rf", RandomForestClassifier(class_weight="balanced", random_state=42))
])

In [53]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

# ---- Fill NA with mode (safer than chained ffill/bfill) ----
mode_md = train["major_discipline"].mode()[0]
train["major_discipline"] = train["major_discipline"].fillna(mode_md)

# ---- Encode categorical values ----
md_le = LabelEncoder()
train["major_discipline"] = md_le.fit_transform(train["major_discipline"])


In [54]:
import re
import numpy as np
import pandas as pd

def _parse_exp(x):
    """Map experience strings to integers:
       <1 -> 0, 1..20 -> 1..20, >20 -> 21. Unrecognized -> NaN"""
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()

    # common variants
    if s in {"<1", "< 1"}:
        return 0
    if s in {">20", "> 20"}:
        return 21
    # sometimes people store '0' or '0.0'
    if s in {"0", "0.0"}:
        return 0

    # plain integers like '1','2',...'20'
    if s.isdigit():
        return int(s)

    # fallback: extract first number if present
    m = re.search(r"\d+", s)
    if m:
        v = int(m.group())
        # clamp into [0,21]
        if v < 1: 
            return 0
        if v > 20:
            return 21
        return v

    # anything else -> NaN
    return np.nan

# ---- Convert to numeric on TRAIN, then impute with TRAIN median ----
train_exp_num = train['experience'].map(_parse_exp).astype('float')
train_med = np.nanmedian(train_exp_num)
train.loc[:, 'experience'] = np.where(np.isnan(train_exp_num), train_med, train_exp_num).astype(int)


In [55]:
import numpy as np

# 1) Normalize ranges -> buckets
bucket_map = {
    '<10':'Startup','10/49':'Small','50-99':'Small',
    '100-500':'Medium','500-999':'Medium',
    '1000-4999':'Large','5000-9999':'Large','10000+':'Large'
}

train_bs = train['company_size'].replace(bucket_map)


# 2) Fill missing using TRAIN stats (no leakage)
mode_bucket = train_bs.mode()[0]
train_bs = train_bs.ffill().bfill().fillna(mode_bucket)


# Optional: see what's still unexpected
# print("Unmapped TRAIN:", set(train_bs.unique()) - {'Startup','Small','Medium','Large'})
# print("Unmapped TEST :", set(test_bs.unique())  - {'Startup','Small','Medium','Large'})

# 3) Force any leftover/unexpected labels to the mode bucket
valid_labels = {'Startup','Small','Medium','Large'}
train_bs = np.where(pd.Series(train_bs).isin(valid_labels), train_bs, mode_bucket)
test_bs  = np.where(pd.Series(test_bs ).isin(valid_labels), test_bs , mode_bucket)

# 4) Ordinal mapping
ord_map = {'Startup': 0, 'Small': 1, 'Medium': 2, 'Large': 3}
train.loc[:, 'company_size'] = pd.Series(train_bs).map(ord_map).astype('int32')
test.loc[:,  'company_size'] = pd.Series(test_bs ).map(ord_map).astype('int32')


In [22]:
train

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,1,Has relevent experience,no_enrollment,Graduate,5,21,1,NaN,1,36,1
1,29725,city_40,0.776,1,No relevent experience,no_enrollment,Graduate,5,15,1,Pvt Ltd,>4,47,0
2,11561,city_21,0.624,1,No relevent experience,Full time course,Graduate,5,5,1,NaN,never,83,0
3,33241,city_115,0.789,1,No relevent experience,NaN,Graduate,1,0,1,Pvt Ltd,never,52,1
4,666,city_162,0.767,1,Has relevent experience,no_enrollment,Masters,5,21,1,Funded Startup,4,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,city_173,0.878,1,No relevent experience,no_enrollment,Graduate,2,14,3,NaN,1,42,1
19154,31398,city_103,0.920,1,Has relevent experience,no_enrollment,Graduate,5,14,3,NaN,4,52,1
19155,24576,city_103,0.920,1,Has relevent experience,no_enrollment,Graduate,5,21,1,Pvt Ltd,4,44,0
19156,5756,city_65,0.802,1,Has relevent experience,no_enrollment,High School,5,0,2,Pvt Ltd,2,97,0


In [59]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

cat_cols = ['company_type',  # + other categoricals
            # ...
           ]
num_cols = [ # your numeric columns
           ]

categorical_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

numeric_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

preprocessor = ColumnTransformer([
    ("cat", categorical_tf, cat_cols),
    ("num", numeric_tf, num_cols)
])

model = Pipeline([
    ("prep", preprocessor),
    ("rf", RandomForestClassifier(class_weight="balanced", random_state=42))
])


In [62]:
import re
import numpy as np
import pandas as pd

def _parse_last_new_job(x):
    """
    Map last_new_job to ordered ints:
      never->0, 1->1, 2->2, 3->3, 4+ (>=4)->4
    Handles messy variants; unknown -> NaN (to be imputed).
    """
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()

    # common 'never' variants
    if s in {"never", "no", "no change", "no-change", "none"}:
        return 0

    # explicit 4+ variants
    if s in {"4+", ">4", "4 plus", "4 or more", "4orMore"}:
        return 4

    # pure digits like '1','2','3','4'
    if s.isdigit():
        v = int(s)
        return 4 if v >= 4 else v

    # fallback: extract first number if present
    m = re.search(r"\d+", s)
    if m:
        v = int(m.group())
        return 4 if v >= 4 else max(0, v)

    return np.nan

# ---- TRAIN: parse -> impute with TRAIN median -> cast int ----
train_lnj_num = train['last_new_job'].map(_parse_last_new_job).astype('float')
train_lnj_med = np.nanmedian(train_lnj_num)   # median in [0..4]
train.loc[:, 'last_new_job'] = np.where(np.isnan(train_lnj_num), train_lnj_med, train_lnj_num).astype('int8')




In [65]:
import pandas as pd
import numpy as np

# 1) Fill NAs (same rule, using TRAIN mode)
mode_city = train['city'].mode()[0]
train_c = train['city'].ffill().bfill().fillna(mode_city)


# 2) Build frequency map on TRAIN only
freq = (train_c.value_counts(normalize=True))  # proportion per city
# Optionally log-scale to compress:
freq = np.log1p(freq)  # comment out if you prefer raw proportions

# 3) Map; unseen in TEST -> smallest seen frequency (or 0.0)
min_freq = float(freq.min()) if len(freq) else 0.0
train.loc[:, 'city'] = train_c.map(freq).fillna(min_freq).astype('float32')



/var/folders/wd/ry1xh_yd375d5mpsb20bqkvw0000gn/T/ipykernel_81829/751673650.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_c = train['city'].ffill().bfill().fillna(mode_city)


In [66]:
import numpy as np
import pandas as pd

# Normalize text (lower/strip); keep original col names
def _norm_re(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().lower()
    if "has" in s: return "has"
    if "no"  in s: return "no"
    return np.nan  # unexpected -> NaN

train_re = train['relevent_experience'].map(_norm_re)


# Fill missing using TRAIN mode (no leakage)
mode_re = train_re.mode()[0] if not train_re.mode().empty else "no"
train_re = train_re.ffill().bfill().fillna(mode_re)


# Map to integers: has -> 1, no -> 0
bin_map = {"has": 1, "no": 0}
train.loc[:, 'relevent_experience'] = train_re.map(bin_map).astype('int8')



In [68]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  19158 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        19158 non-null  int64  
 8   experience              19158 non-null  object 
 9   company_size            19158 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            19158 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  int64  
dtypes: float64(1), int64(4), object(9)
mem

In [69]:
train.head()


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,0.204833,0.920,Male,1,no_enrollment,Graduate,5,21,1,NaN,1,36,1
1,29725,0.003543,0.776,Male,0,no_enrollment,Graduate,5,15,1,Pvt Ltd,4,47,0
2,11561,0.131938,0.624,Male,0,Full time course,Graduate,5,5,1,NaN,0,83,0
3,33241,0.002815,0.789,Male,0,NaN,Graduate,1,0,1,Pvt Ltd,0,52,1
4,666,0.019846,0.767,Male,1,no_enrollment,Masters,5,21,1,Funded Startup,4,8,0


In [31]:
import numpy as np
import pandas as pd

def norm_re(x):
    # pass through numeric 0/1
    if pd.notna(x) and isinstance(x, (int, np.integer, float)) and x in (0, 1):
        return int(x)
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()
    if "has" in s:
        return 1
    if "no" in s:
        return 0
    return np.nan

def safe_mode(s: pd.Series, fallback):
    m = s.mode(dropna=True)
    return m.iloc[0] if len(m) else fallback

# apply robust mapping
re_tr = train['relevent_experience'].map(norm_re)
re_te = test['relevent_experience'].map(norm_re)

# pick a safe fallback (0 = "no") if the mode is empty
re_mode = safe_mode(re_tr, fallback=0)

train['relevent_experience'] = re_tr.ffill().bfill().fillna(re_mode).astype('int64')
test['relevent_experience']  = re_te.ffill().bfill().fillna(re_mode).astype('int64')


In [32]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  int64  
 2   city_development_index  19158 non-null  float64
 3   gender                  19158 non-null  int64  
 4   relevent_experience     19158 non-null  int64  
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        19158 non-null  object 
 8   experience              19158 non-null  object 
 9   company_size            19158 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            19158 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  int64  
dtypes: float64(1), int64(6), object(7)
mem

In [34]:
import pandas as pd
import numpy as np

# Bucketing dict (raw -> bucket)
bucket_map = {
    '<10':'Startup','10/49':'Small','50-99':'Small',
    '100-500':'Medium','500-999':'Medium',
    '1000-4999':'Large','5000-9999':'Large','10000+':'Large'
}
ord_map = {'Startup':0,'Small':1,'Medium':2,'Large':3}
valid = set(ord_map.keys())

# 1) Work with explicit string dtype to avoid the FutureWarning
tr_raw = train['company_size'].astype('string')
te_raw = test['company_size'].astype('string')

# 2) Normalize/bucket raw ranges
tr_bs = tr_raw.replace(bucket_map)
te_bs = te_raw.replace(bucket_map)

# 3) Choose a safe mode from TRAIN; fall back to 'Small' if empty
mode_bucket = tr_bs.dropna().mode()
mode_bucket = (mode_bucket.iloc[0] if len(mode_bucket) else 'Small')

# 4) Force unexpected labels to the mode bucket (and fill NaNs)
tr_bs = tr_bs.where(tr_bs.isin(valid), mode_bucket).fillna(mode_bucket)
te_bs = te_bs.where(te_bs.isin(valid), mode_bucket).fillna(mode_bucket)

# 5) Map to ordered ints, fill any leftover NaNs, then cast
train.loc[:, 'company_size'] = tr_bs.map(ord_map).fillna(ord_map[mode_bucket]).astype('int64')
test.loc[:,  'company_size'] = te_bs.map(ord_map).fillna(ord_map[mode_bucket]).astype('int64')


KeyError: '3'